# Investigation of faceting in the SKA Performance Model
## This notebook provides the plots for the TCC memo TCC-SDP-151123-1-1
## First, do the necessary set-up (run the code block below, first)

In [ ]:
""" These first few lines below import the IPython definitions and methods that we will use. 
Please Refer to ipython_api.py for the implementation """
from ipywidgets import interact, interact_manual, fixed
from api_ipython import SkaIPythonAPI as iapi
from parameter_definitions import *
from parameter_definitions import Constants as c
from implementation import Implementation as imp, PipelineConfig
# Do plotting inline since sending to a specific file in jupyter/matplotlib is fiddly
%matplotlib inline


"""
# NB Recent versions of IPython (4.x.x) require that ipywidgets be installed separately for the 
import lines below to work. Additionally, IPython.html has been deprecated. Only kept here for 
backward-compatibility (for the time being)
"""
#from IPython.html.widgets import interact, interactive, fixed
#from IPython.html.widgets.interaction import interact_manual
#from IPython.html import widgets

# The lines below define the available telescopes, bands and modes that may be chosen.

available_teles = (Telescopes.SKA1_Low, Telescopes.SKA1_Low_old, Telescopes.SKA1_Mid, Telescopes.SKA1_Mid_old, 
                   Telescopes.SKA1_Sur_old, Telescopes.SKA2_Low, Telescopes.SKA2_Mid)
available_bands = (Bands.Low, 
                   Bands.Mid1, Bands.Mid2, Bands.Mid5A, Bands.Mid5B, 
                   Bands.Sur1)
available_modes     = (ImagingModes.Continuum, ImagingModes.Spectral, ImagingModes.FastImg)
available_modes_all = (ImagingModes.Continuum, ImagingModes.Spectral, ImagingModes.FastImg, ImagingModes.All)
verbose_display = True
# Scale the predict step according to the facet size not the FOV : See TCC-SDP-151123-1-1
scale_predict_by_facet = True 


## Get summaries

In [ ]:
verbosity='Overview'

for telescope, band in [(Telescopes.SKA1_Low, Bands.Low),
                        (Telescopes.SKA1_Mid, Bands.Mid1),
                        (Telescopes.SKA1_Mid, Bands.Mid2)]:
    iapi.compare_telescopes_default(telescope, band, ImagingModes.Continuum, \
                                    telescope, band, ImagingModes.Spectral, \
                                    scale_predict_by_facet=scale_predict_by_facet, \
                                    verbosity=verbosity)



## Evaluate Rflop, Rio, Mw_cache as function of tsnap and nfacet over all combinations

In [ ]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np

expressions = ['Rflop', 'Rio', 'Rinterfacet', 'Mw_cache', 'Rfft_predict', 'Rfft_backward']
tsnaps=np.arange(10,3600,60)
nfacets=range(1,15)

verbose = False

display(HTML('<font color="blue">Computing the result -- this may take several (tens of) seconds.</font>'))
for telescope in [Telescopes.SKA1_Mid, Telescopes.SKA1_Low]:
    for immode in [ImagingModes.Continuum, ImagingModes.Spectral, ImagingModes.FastImg]:
        if telescope==Telescopes.SKA1_Low:
            band=Bands.Low
        else:
            band=Bands.Mid1

        for expression in expressions:
            cfg = PipelineConfig(telescope=telescope, mode=immode, band=band,
                                 scale_predict_by_facet=scale_predict_by_facet)
            (p1_values, p2_values, results) = iapi.eval_param_sweep_2d_noopt(cfg, 
                                                                       expression=expression,  
                                                                       tsnaps=tsnaps, nfacets=nfacets,
                                                                       verbose=verbose)
            ptitle = 'Log(%s_peta) %s %s %s %s' % (expression, telescope, immode, band, scale_predict_by_facet)
            iapi.plot_2D_surface(ptitle, p2_values, p1_values, np.log10(results/1e15), ylabel='Tsnap', 
                                 xlabel='Nfacet', nlevels=100)


## Evaluate Rflop, Rio, Mw_cache as function of nfacet (tsnap optimised) over all combinations

In [ ]:
from IPython.display import display, HTML

parameter = 'Nfacet'
param_val_min =1
param_val_max = 7
number_steps = 6

for telescope in [Telescopes.SKA1_Low, Telescopes.SKA1_Mid]:
    if telescope==Telescopes.SKA1_Low:
        band=Bands.Low
    else:
        band=Bands.Mid1
    for immode in [ImagingModes.Continuum]:
        for expression in [ 'Rflop', 'Rio', 'Rinterfacet', 'Mw_cache']:
            print telescope, immode, expression
            display(HTML('<font color="blue">Computing the result -- this may take several (tens of) seconds.</font>'))
            cfg = PipelineConfig(telescope=telescope, mode=immode, band=band,
                                 scale_predict_by_facet=scale_predict_by_facet)
            (param_values, results) = iapi.eval_param_sweep_1d(cfg,
                                                               expression=expression, parameter=parameter, param_val_min=param_val_min,
                                                               param_val_max=param_val_max, number_steps=number_steps, 
                                                               verbose=False)
            header = 'Plotting %s for %s in %s mode as a function of %s' % (expression, telescope, immode, parameter)
            iapi.plot_line_datapoints(header, param_values, np.array(results)/1e15, xlabel=parameter, ylabel=expression)